In [21]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
import pandas as pd

#my helper functions from ipynb file
from ipynb.fs.full.PostgreHelper import * 

# Extract

In [27]:
#my sample google sheet
sheet_id = '1xo1_DZEVBiVz39S5Ln3CSnagbPE78cvBDKtwqtC3hKI'

#extract all tables from the sheet
student_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Students"
df_student = pd.read_csv(student_url)

st_course_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Student_Courses"
df_st_course = pd.read_csv(st_course_url)

course_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Courses_Info"
df_course_info = pd.read_csv(course_url)

payment_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Payment_Method"
df_payment = pd.read_csv(payment_url)

purchased_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Purchased_Info"
df_purchase = pd.read_csv(purchased_url)

In [29]:
df_student.head()

,student_id,first_name,last_name,full_name,phone,email,address,birthyear,nationality
0,S001,student,1,student001,97001,student001@smail.com,Yangon,1990,Myanmar
1,S002,student,2,student002,97002,student002@smail.com,Yangon,1990,Myanmar
2,S003,student,3,student003,97003,student003@smail.com,Yangon,1990,Myanmar
3,S004,student,4,student004,97004,student004@smail.com,Yangon,1990,Myanmar
4,S005,student,5,student005,97005,student005@smail.com,Yangon,1990,Myanmar


In [30]:
df_st_course.head()

,student_id,course_id,status,completed date,feedback
0,S001,COURSE001,Completed,2/3/2022,feedback001
1,S002,COURSE002,Completed,2/1/2022,feedback002
2,S003,COURSE003,In progress,NaN,feedback003
3,S003,COURSE004,In progress,NaN,feedback004
4,S004,COURSE004,Completed,2/3/2022,feedback005


In [31]:
df_course_info.head()

,course_id,course_category,course_duration,course_level,instructor,course_price
0,COURSE001,Programming,4 hrs,Begineer,instructor001,50
1,COURSE002,Programming,4 hrs,Intermediate,instructor002,50
2,COURSE003,Programming,4 hrs,Advanced,instructor003,50
3,COURSE004,Programming,4 hrs,Begineer,instructor004,50
4,COURSE005,Programming,4 hrs,Intermediate,instructor005,50


In [32]:
df_payment.head()

,payment_method_id,student_id,payment_type,account_no,active
0,P001,S001,visa,1,yes
1,P002,S002,paypal,2,yes
2,P003,S003,master,3,yes
3,P004,S004,visa,4,yes
4,P005,S005,paypal,5,yes


In [33]:
df_purchase.head()

,student_id,course_id,payment_method_id,purchased_date
0,S001,COURSE001,P001,10/4/2021
1,S002,COURSE002,P002,10/5/2021
2,S003,COURSE003,P003,10/6/2021
3,S004,COURSE004,P004,10/7/2021
4,S005,COURSE005,P005,10/8/2021


# Transform (Dimensional Model)

In [ ]:
#there are many analysis questions for student online courses data
#for examples, which course has highest completion percentage?
#which course has good overall feedbacks
#let's create a fact table including students and courses information

# Load

In [ ]:
#connect to my Postgre database that hosted on ElephantSql with free instance
connection, cursor = connect(host = 'arjuna.db.elephantsql.com', 
                             database = '*******', 
                             user = '*******', 
                             password = '*******', 
                             port = '5432')

In [ ]:
#create new table for the above youtube data
query = 'CREATE TABLE youtube_channel \
        (id SERIAL PRIMARY KEY, title text, url text, liked int, disliked int, views int, comment_count int); '
create_table(connection, cursor, query)

In [ ]:
#inserting the dataframe into youtube_channel table

#convert dataframe in tuples to iterate
tuples = [tuple(x) for x in df.to_numpy()]

#df columns into string
cols = ','.join(list(df.columns))

#insert query
query = "INSERT INTO youtube_channel(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s)" % (cols)

#inserting with executemany function
try:
    cursor.executemany(query, tuples)
    print("Data inserted using execute_many() successfully!")
except (Exception, psycopg2.DatabaseError) as err:
    print(err)
    cursor.close()

In [ ]:
#read PostgreSQL table into pandas dataframe
data = pd.read_sql_query(con = connection, sql = 'SELECT * FROM youtube_channel')
data.head(10)